# Group 10 - First Year Project 
### Data Science at ITU 
## Project 1 - Road collisions analysis 2021 

The first project of Data Science's second semester required us to inform various city leaders around the United Kingdom about traffic fatalities and injuries; and give some insight for urban transport planning. This jupyter notebook presents Group 10's exploration of 2019 UK road collision data with complimentary analysis, visualisation and findings with relation to the city of Sheffield.


# Task 0 - Data filtering and cleaning

## Loading data
Our first task invovles loading our 3 csv files (downloaded from https://data.gov.uk/dataset/road-accidents-safety-data on the 4th January 2021) into a traversable data structure we can start working with; such as a dictionary. We will then need to start the process of filtering and cleaning our data.

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import math
from scipy.stats import chi2_contingency

In [ ]:
raw_data = {}

TITLES = ['accidents', 'casualties', 'vehicles'] # Capitalisation is the standard for constants

Pandas has a convinient method for reading in csv files and storing the data in a DataFrame structure offers flexibility.

In [ ]:
raw_data['accidents'] = pd.read_csv("../data/raw/Road Safety Data - Accidents 2019.csv", dtype = None, encoding='utf-8-sig')
raw_data['casualties'] = pd.read_csv("../data/raw/Road Safety Data - Casualties 2019.csv", dtype = None, encoding='utf-8-sig')
raw_data['vehicles'] = pd.read_csv("../data/raw/Road Safety Data- Vehicles 2019.csv", dtype = None, encoding='utf-8-sig') # File name missing a space before hyphen 

In [ ]:
for TITLE in TITLES:
    print(TITLE + "\tDataFrame shape: " + str(raw_data[TITLE].shape))

 ### Data type insight?    !!! Not finished !!!

In [ ]:
#todo #DO IT FASTER

## Locating our city
With our data loaded into a dictionary containing DataFrames, we needed to locate the accidents that occured in Sheffield. To do this we made a mask identifying which accidents occured in the local district authority 215, which is the district of Sheffield.


In [ ]:
sheffield_mask = (raw_data['accidents']["Local_Authority_(District)"] == 215)

In [ ]:
sheffield_data = {}

sheffield_data['accidents'] = raw_data['accidents'][sheffield_mask]

# Viewing the first 5 rows in the dataframe
sheffield_data['accidents'][:5]

In [ ]:
sheffield_data['accidents'].shape

We can now see that there are 909 records of road traffic accidents occuring in Sheffield over 2019. The next step was using the primary key (accident_index) of the accidents to find the accidents in Sheffield that each casualty and vehicle linked to, in order to complete our sheffield_data dictionary.

In [ ]:
sheffield_data['accidents']["Accident_Index"][:5]

In [ ]:
for TITLE in TITLES[1:]: 
    sheffield_data[TITLE] = raw_data[TITLE][raw_data[TITLE]['Accident_Index'
                                                           ].isin(sheffield_data["accidents"]["Accident_Index"])]

We can now find out the number of rows and columns in our Sheffield data set

In [ ]:
for TITLE in TITLES:
    print("Sheffield " + TITLE + "\tDataFrame shape: " + str(sheffield_data[TITLE].shape))

There is 909 accidents each with 32 attributes.
There is 1191 casualties that occured in the 909 accidents each with 16 attributes.
There is 1648 vehicles that occured in the 909 accidents each with 23 attributes.

This indicates that some accidents involved more than one car, which logically makes sense. If there were a 1:1 ratio between accidents and vehicles, all the vehicles had driven into a object and not caused another vehicle to be part of the accident.

Creating the data files containing only the data from Sheffield

In [ ]:
sheffield_data["accidents"].to_csv("../data/processed/sheffield_accidents.csv")
sheffield_data["vehicles"].to_csv("../data/processed/sheffield_vehicles.csv")
sheffield_data["casualties"].to_csv("../data/processed/sheffield_casualties.csv")

## Relational data table model 
In terms of the structure of the three databases or data tables, the accidents table's primary key is used as a foreign key by both the casualties and vehicles data tables, creating a one-to-many link between the accidents and both the casualties and vehicles tables. From this data table structure, we can infer that multiple casualties and vehicles can be involved in a single accident; which makes sense logically.

## Sanity check
Performing a sanity check on our Sheffield road accident data is a critical first step before we can begin to think about further analysing or drawing conclusions from the data. We may also have to sanity check all the data if we wish to draw conclusions therefrom.

In [ ]:
#Check for empty strings in categorical fields  
#Check for NULL values                          #DONE more efficient method probably possible with michaels method (command line)
#Check duplicate records                        #DONE
#Check non-linked records in secondary tables   

### Checking for null values in our road accident dataset

In [ ]:
flag = False
for TITLE in TITLES:
    if raw_data[TITLE].isnull().values.any():
        for column in list(raw_data[TITLE].columns):
            nulls = sum(raw_data[TITLE][column].isnull())
            if sum(raw_data[TITLE][column].isnull()):
                print(TITLE + " - " + column + "\t has " + str(nulls) + " null values")
                flag = True
if not flag: print("There are no null values in the dataset")

### Checking for null values in our sheffield dataset

In [ ]:
flag = False
for TITLE in TITLES:
    if sheffield_data[TITLE].isnull().values.any():
        for column in list(sheffield_data[TITLE].columns):
            nulls = sum(sheffield_data[TITLE][column].isnull())
            if sum(sheffield_data[TITLE][column].isnull()):
                print(TITLE + " - " + column + "\t has " + str(nulls) + " null values")
                flag = True
if not flag: print("There are no null values in the dataset")

### Checking for repeated indexes in our sheffield dataset

In [ ]:
if len(sheffield_data["accidents"]["Accident_Index"]) == len((sheffield_data["accidents"]["Accident_Index"].unique())):
    print("No repeated indexes")
else: print("Repeated indexes detected")

In [ ]:
###NOT FULLY WORKING
sheffield_data["accidents"].replace(-1, np.nan, inplace=True)
missingdata_df = sheffield_data["accidents"].columns[sheffield_data["accidents"].isnull().any()].tolist()
msno.matrix(sheffield_data["accidents"][missingdata_df]);

In [ ]:
# ###NOT FULLY WORKING
sheffield_data['vehicles'].replace(-1, np.nan, inplace=True)
missingdata_df = sheffield_data['vehicles'].columns[sheffield_data['vehicles'].isnull().any()].tolist()
msno.matrix(sheffield_data['vehicles'][missingdata_df]);

In [ ]:
###NOT FULLY WORKING
sheffield_data['casualties'].replace(-1, np.nan, inplace=True)
missingdata_df = sheffield_data['casualties'].columns[sheffield_data['casualties'].isnull().any()].tolist()
msno.matrix(sheffield_data['casualties'][missingdata_df]);

## Numerical summary

In [ ]:
import matplotlib.pyplot as plt

### Five Number Summary 

### Box Plots

#### Number of vehicles in an accident and age of vehicle

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

mask_num_vehicle = (sheffield_data['accidents']['Number_of_Vehicles']>0)
NumVehicles = sheffield_data['accidents']["Number_of_Vehicles"][mask_num_vehicle]
NumVehicles = NumVehicles.to_frame()
NumVehicles.describe(), NumVehicles.boxplot(ax=ax[0]);

mask_age_vehicle = (sheffield_data['vehicles']['Age_of_Vehicle']>=0)
AgeVehicles = sheffield_data['vehicles']['Age_of_Vehicle'][mask_age_vehicle]
AgeVehicles = AgeVehicles.to_frame()
AgeVehicles.describe(), AgeVehicles.boxplot(ax=ax[1]);

#### Age of driver

In [ ]:
#AGE OF DRIVER
mask_age = (sheffield_data['vehicles']['Age_of_Driver']>=16)
Age_Driver =  sheffield_data['vehicles']["Age_of_Driver"][mask_age]
Age_Driver = Age_Driver.to_frame()
Age_Driver.describe(), Age_Driver.boxplot();

#### Age of male and female drivers

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

mask_male = (sheffield_data['vehicles']['Sex_of_Driver']==1)
Age_MaleDrivers = sheffield_data['vehicles']["Age_of_Driver"][mask_age][mask_male]
Age_MaleDrivers = Age_MaleDrivers.to_frame()
Age_MaleDrivers.describe(), Age_MaleDrivers.boxplot(ax=ax[0])

mask_female = (sheffield_data['vehicles']['Sex_of_Driver']==2)
Age_FemaleDrivers = sheffield_data['vehicles']["Age_of_Driver"][mask_age][mask_female]
Age_FemaleDrivers = Age_FemaleDrivers.to_frame()
Age_FemaleDrivers.describe(), Age_FemaleDrivers.boxplot(ax=ax[1])
ax[0].set_ylabel("Age"), ax[1].set_ylabel("Age");

#### Age of casualty

In [ ]:
mask_age2 = (sheffield_data['casualties']['Age_of_Casualty']>=0)
AgeCasualty = sheffield_data['casualties']["Age_of_Casualty"][mask_age2]
AgeCasualty = AgeCasualty.to_frame()

AgeCasualty.describe(), AgeCasualty.boxplot();

### Frequency Histogram

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))
mask = (sheffield_data["vehicles"]["Age_of_Driver"] >=0)
sheffield_data["accidents"]["Number_of_Vehicles"].hist(ax=ax[0,0], bins=20, bottom=0.1,color = 'silver',edgecolor = 'black')
sheffield_data["accidents"]["Speed_limit"].hist(ax=ax[1,1], bins=7,color = 'silver',edgecolor = 'black');
sheffield_data["accidents"]["Number_of_Casualties"].hist(ax=ax[0,1], bins=20, bottom=0.1,color = 'silver',edgecolor = 'black')
sheffield_data["vehicles"]["Age_of_Driver"][mask].hist(ax=ax[1,0], bins=50, bottom=0.1,color = 'silver',edgecolor = 'black')

ax[0,0].set_title('Histogram over Number of Vehicles in an accident')
ax[1,1].set_title('Histogram over Speed Limit in an accident')
ax[0,1].set_title('Histogram over Number of Casualties in an accident')
ax[1,0].set_title('Histogram over Age of Driver in an accident')

ax[0,0].set_ylabel('Frequency')
ax[1,0].set_ylabel('Frequency')
ax[0,1].set_ylabel('Frequency')
ax[1,1].set_ylabel('Frequency')

ax[0,0].set_xlabel('Amount of vehicles')
ax[1,1].set_xlabel('Speed Limit')
ax[0,1].set_xlabel('Number of Casualties')
ax[1,0].set_xlabel('Age of Driver')

ax[0,0].set_yscale('log')
ax[1,1].set_yscale('log')
ax[0,1].set_yscale('log')
ax[1,0].set_yscale('log')
plt.plot();

### Frequency Barplot

In [ ]:
AS_count = sheffield_data["accidents"]["Accident_Severity"].value_counts()
AS_unq = sheffield_data["accidents"]["Accident_Severity"].unique()

RT = sheffield_data["accidents"]["Road_Type"][(sheffield_data["accidents"]["Road_Type"] >0)]
RT_count = RT.value_counts()
RT_unq = [1,2,3,4,5,6]

sex = sheffield_data["vehicles"]["Sex_of_Driver"][(sheffield_data["vehicles"]["Sex_of_Driver"] >0)]
sex_counts = sex.value_counts()
sex_unq = sex.unique()

CC_counts = sheffield_data["casualties"]["Casualty_Class"].value_counts()
CC_unq = sheffield_data["casualties"]["Casualty_Class"].unique()

mask = (sheffield_data["vehicles"]["1st_Point_of_Impact"] >= 0)
PoI_counts = sheffield_data["vehicles"]["1st_Point_of_Impact"][mask].value_counts()
PoI_unq = sheffield_data["vehicles"]["1st_Point_of_Impact"][mask].unique()


In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(15, 15))

ax[0,0].set_title("Accident Severity Frequency Barplot")
ax[0,0].set_ylabel("Frequency")
ax[0,0].bar(sorted(AS_unq), sorted(AS_count), tick_label = ["Fatal", "Serious", "Slight"],color = 'silver',edgecolor = 'black')

ax[1,0].set_title("Road Type Frequency Barplot")
ax[1,0].set_ylabel("Road Type")
ax[1,0].set_xlabel("Frequency")
l = ["Roundabout", "One way street", "Dual carriageway","Single carriageway", "Slip road", "Unknown"]  
ax[1,0].barh(sorted(RT_unq), sorted(RT_count), tick_label = l,color = 'silver',edgecolor = 'black')

                                                                                                                        


ax[0,1].set_title("Sex Frequency Barplot")
ax[0,1].set_ylabel("Frequency")
ax[0,1].bar(sorted(sex_unq), sex_counts, tick_label = ["Male","Female","Not Known"],color = 'silver',edgecolor = 'black')


ax[1,1].set_title("Casualty Class Frequency Barplot")
ax[1,1].set_ylabel("Frequency")
ax[1,1].bar(sorted(CC_unq), CC_counts, tick_label = ["Driver or rider", "Passenger", "Pedestrian"],color = 'silver',edgecolor = 'black')


ax[2,0].set_title("1st Point of Impact Frequency Barplot")
ax[2,0].set_ylabel("Frequency")
ax[2,0].bar(PoI_unq, PoI_counts, tick_label = ["Did not impact", "Front", "Back","Offside", "Nearside"], color = 'silver',edgecolor = 'black')


field_name = "Light_Conditions"
field_categories = {1: "Daylight", 4: "Darkness - lights lit", 5: "Darkness - lights unlit", 6: "Darkness - no lighting", 7: "Darkness - lighting unknown", -1: "Data missing or out of range"}

ax[2,1].set_title(field_name)
ax[2,1].set_ylabel("Frequency")

categories, counts = np.unique(sheffield_data["accidents"][field_name], return_counts=True)
ax[2,1].set_xticks(range(len(categories)))
ax[2,1].set_xticklabels(field_categories.values());

ax[2,1].bar(range(len(categories)), counts, color = 'silver',edgecolor = 'black')

plt.setp(ax[2,1].get_xticklabels(), rotation=30, ha='right')

plt.plot();

# Task 1 - Single variable analysis


### Report the frequency of road collisions in your city for 

1. different age groups 
2. in different times (during the day, the week, or year)
3. and for differences in one other condition.


#### Different age groups

In [ ]:
mask_age = (sheffield_data["vehicles"]["Age_of_Driver"] >= 0)
AoD = sheffield_data["vehicles"]["Age_of_Driver"][mask_age]


l = []
mask = (sheffield_data["vehicles"]["Age_of_Driver"] <20)

l.append(AoD[mask].shape[0])
mask = (sheffield_data["vehicles"]["Age_of_Driver"] >20) & (sheffield_data["vehicles"]["Age_of_Driver"] <40)
l.append(AoD[mask].shape[0])
mask = (sheffield_data["vehicles"]["Age_of_Driver"] >40) & (sheffield_data["vehicles"]["Age_of_Driver"] <60)
l.append(AoD[mask].shape[0])
mask = (sheffield_data["vehicles"]["Age_of_Driver"] >60) & (sheffield_data["vehicles"]["Age_of_Driver"] <80)
l.append(AoD[mask].shape[0])
mask = (sheffield_data["vehicles"]["Age_of_Driver"] >80)
l.append(AoD[mask].shape[0])

fig = plt.figure(figsize=(15,5))
axes = fig.add_axes([0,0,1,1])
axes.set_title("Frequency of road collisions in Sheffield for different age groups")
axes.set_ylabel("Frequency")
interval = ["<20",'20-40', '40-60', '60-80', '80+']
axes.bar(interval, l,color = 'silver',edgecolor = 'black')

for index,value in enumerate(l):
    axes.text(index - 0.015, value + 0.05,str(value),ha='center', va='bottom')

plt.plot();

#### Different times

##### Day

In [ ]:
l = [['00:00', '02:00'], 
     ['02:00', '04:00'], 
     ['04:00', '06:00'], 
     ['06:00', '08:00'], 
     ['08:00', '10:00'], 
     ['10:00', '12:00'],
     ['12:00', '14:00'], 
     ['14:00', '16:00'], 
     ['16:00', '18:00'], 
     ['18:00', '20:00'], 
     ['20:00', '22:00'],
     ['22:00', '24:00']]

empty = []
for i in range(0,12):
    mask = (sheffield_data["accidents"]["Time"] < l[i][1]) & (sheffield_data["accidents"]["Time"] > l[i][0])
    total = len(sheffield_data["accidents"][mask]["Time"])
    empty.append(total)
    
fig = plt.figure(figsize=(15,5))
axes = fig.add_axes([0,0,1,1])
axes.set_title("Frequency of road collisions in Sheffield in a 2 hour interval")
axes.set_ylabel("Frequency")
interval = ["00:00-02:00",'02:00-04:00', '04:00-06:00', '06:00-08:00', '08:00-10:00', '10:00-12:00',
                 '12:00-14:00','14:00-16:00', '16:00-18:00', '18:00-20:00', '20:00-22:00', '22:00-24:00']
axes.bar(interval, empty,color = 'silver',edgecolor = 'black');
for index,value in enumerate(empty):
    axes.text(index - 0.015, value + 0.05,str(value),ha='center', va='bottom')

##### Week

In [ ]:
week = ["Monday", "Tuesday", "Wednesday","Thursday","Friday", "Saturday", "Sunday"]
unq = sheffield_data["accidents"]["Day_of_Week"].unique()
count = sheffield_data["accidents"]["Day_of_Week"].value_counts()
fig = plt.figure(figsize=(8, 5))
axes = fig.add_axes([0, 0, 1, 1])
axes.set_title("Accidents each day from 01/01/2019 to 31/12/2019 in Sheffield")
axes.set_ylabel("Total number of accidents")
axes.bar(sorted(unq), count, tick_label = week,color = 'silver',edgecolor = 'black')

for index,value in enumerate(count):
    axes.text(index + 1 - 0.015, value + 0.05,str(value),ha='center', va='bottom')
    

plt.plot();

##### Year

In [ ]:
list_month = [
'January',
'February',
'March',
'April',
'May',
'June',
'July',
'August',
'September',
'October',
'November',
'December'
]
accident_severity = sheffield_data['accidents']['Accident_Severity']
accident_dates = pd.to_datetime(sheffield_data['accidents']['Date'])
a = accident_severity.groupby(accident_dates.dt.month).sum()
fig,ax = plt.subplots(figsize = (16,4))
a.plot.bar(color = 'silver',edgecolor = 'black',rot = 0)
ax.set_title('Monthly Accident Numbers')
ax.set_xlabel('Months')
ax.set_ylabel('Number of Accidents')
ax.set_xticklabels(list_month)
for index,value in enumerate(a):
    ax.text(index - 0.015, value + 0.05,str(value),ha='center', va='bottom')

##### Number of accidents each day   

In [ ]:
#COUNTING THE ACCIDENTS EACH DAY IN EVERY MONTH

accident_severity = sheffield_data['accidents']['Accident_Severity']

accident_dates = pd.to_datetime(sheffield_data['accidents']['Date'])

accident_dates1 = accident_dates.dt.date



mask = (accident_dates > "2019-12-01") & (accident_dates <= "2019-12-31")
s = accident_dates1[mask].value_counts()
s1 = pd.DataFrame(s)
s1.columns = ['Accidents']
print(s1.to_markdown(tablefmt = 'github'))

##### Monthly Accident Severity

In [ ]:
#####MONTHLY SEVERITY 2
accident_severity = sheffield_data['accidents']['Accident_Severity']
accident_dates = pd.to_datetime(sheffield_data['accidents']['Date'])
#accident_severity.groupby(accident_dates.dt.month).count().plot(kind="bar")
a = accident_severity.groupby(accident_dates.dt.month)
a1 = a.value_counts()

fig,ax = plt.subplots(figsize = (16,4))
a1.plot.bar(color = 'silver',edgecolor = 'black',rot = 45)
ax.set_title('Monthly Accident Severity')
ax.set_xlabel('(Month,Accident_Severity)')
ax.set_ylabel('Number of Accidents')
ax.legend(loc = "upper left")
for index,value in enumerate(a1):
    ax.text(index - 0.015, value + 0.05,str(value),ha='center', va='bottom')

# Task 2 - Associations


## Research whether there is a significant statistical association in your city, either
1. between some vehicle attributes and accident circumstances, or 
2. between some casualty attributes and accident circumstances. 

Report whether there is a statistically significant association between such variables or not, together with the appropriate statistical metric(s).
 
Discuss why this association, or the lack of
this association, is relevant for urban/transport planning.

### Speed limit and Accident Severity

In [ ]:
mask = (sheffield_data["accidents"]["Speed_limit"] != -1)
severityspeed = np.array([sheffield_data["accidents"]["Speed_limit"][mask], sheffield_data["accidents"]["Accident_Severity"][mask]]).T
observed_pd = pd.crosstab(severityspeed[:, 0], severityspeed[:, 1], rownames = ["Speed limit"], colnames = ["Accident Severity"]) 
observed_pd_n = pd.crosstab(severityspeed[:, 0], severityspeed[:, 1], rownames = ["Speed limit"], colnames = ["Accident Severity"], normalize = True) 
observed = observed_pd.to_numpy()

chiVal, pVal, df, expected = chi2_contingency(observed)

alpha = 0.05
print("p value is " + str(pVal)) 
if pVal <= alpha: 
    print('Dependent (reject H0)') 
else: 
    print('Independent (H0 holds true)')
    
N = observed.sum(axis = 1).sum()
    
V = np.sqrt( (chiVal/N) / (min(observed.shape)-1) )
print("V: "+ str(V) + " a value close to 0 means weak, close to 1 means strong")
#observed_pd

Since pVal > 0.1 the association between speed limit and accident severity in Sheffield is not significant. 

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))

severity_labels = ["Fatal", "Serious", "Slight"]
speed_categories = {20: "20 MPH", 30: "30 MPH", 40: "40 MPH", 50: "50 MPH", 60: "60 MPH", 70: "70 MPH"}
x = np.array(list(speed_categories.keys()))

for i, ax in enumerate(axes[0]):
    ax.plot(x, observed[:,i], 'ro-', label='Observed')
    ax.plot(x, expected[:,i], 'bo-', label='Expected')
    if i==0: 
        ax.set_ylabel('Casualties')
        ax.legend(loc='best');
    ax.set_title(severity_labels[i])
    ax.set_xticks(x)
    ax.set_xticklabels(list(speed_categories.values()))
    fig.autofmt_xdate(rotation=45)

for i, ax in enumerate(axes[1]):
    ax.plot(x, observed[:,i]/expected[:,i], 'go-')
    ax.plot(x, np.ones(x.shape), 'k:')
    
    if i==0: 
        ax.set_ylabel('Observed/Expected')
    ax.set_xticks(x)
    ax.set_xticklabels(list(speed_categories.values()))
    fig.autofmt_xdate(rotation=45)

### Accident severity and road type

In [ ]:
mask = (sheffield_data["accidents"]["Road_Type"] != -1)
severityRoad = np.array([sheffield_data["accidents"]["Road_Type"][mask], sheffield_data["accidents"]["Accident_Severity"][mask]]).T
observed_pd = pd.crosstab(severityRoad[:, 0], severityRoad[:, 1], rownames = ["Road Type"], colnames = ["Accident Severity"]) 
observed_pd_n = pd.crosstab(severityRoad[:, 0], severityRoad[:, 1], rownames = ["Road Type"], colnames = ["Accident Severity"], normalize = True) 
observed = observed_pd.to_numpy()

chiVal, pVal, df, expected = chi2_contingency(observed)

alpha = 0.05
print("p value is " + str(pVal)) 
if pVal <= alpha: 
    print('Dependent (reject H0)') 
else: 
    print('Independent (H0 holds true)')
    
N = observed.sum(axis = 1).sum()    

V = np.sqrt( (chiVal/N) / (min(observed.shape)-1) )
print("V: "+ str(V) + " a value close to 0 means weak, close to 1 means strong")
#observed_pd

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 6))

severity_labels = ["Fatal", "Serious", "Slight"]
road_categories = {1: "Roundabout", 2: "One way street", 3: "Dual carriageway", 6: "Single carriageway", 7: "Slip road", 9: "Unknown"}
x = np.array(list(road_categories.keys()))

for i, ax in enumerate(axes[0]):
    ax.plot(x, observed[:,i], 'ro-', label='Observed')
    ax.plot(x, expected[:,i], 'bo-', label='Expected')
    if i==0: 
        ax.set_ylabel('Accidents')
        ax.legend(loc='best');
    ax.set_title(severity_labels[i])
    ax.set_xticks(x)
    ax.set_xticklabels(list(road_categories.values()))
    fig.autofmt_xdate(rotation=45)

for i, ax in enumerate(axes[1]):
    ax.plot(x, observed[:,i]/expected[:,i], 'go-')
    ax.plot(x, np.ones(x.shape), 'k:')
    
    if i==0: 
        ax.set_ylabel('Observed/Expected')
    ax.set_xticks(x)
    ax.set_xticklabels(list(road_categories.values()))
    fig.autofmt_xdate(rotation=30)

### Road Type and Light Conditions

In [ ]:
mask = (sheffield_data["accidents"]["Road_Type"] != -1
       ) & (sheffield_data["accidents"]["Light_Conditions"] != -1
       ) & (sheffield_data["accidents"]["Light_Conditions"] != 7)
lightRoad = np.array([sheffield_data["accidents"]["Road_Type"][mask], 
                      sheffield_data["accidents"]["Light_Conditions"][mask]]).T

observed_pd = pd.crosstab(lightRoad[:, 0], lightRoad[:, 1], 
                          rownames = ["Road Type"], colnames = ["Light Conditions"]) 

observed_pd_n = pd.crosstab(lightRoad[:, 0], lightRoad[:, 1], 
                            rownames = ["Road Type"], colnames = ["Light Conditions"], normalize = True) 

observed = observed_pd.to_numpy()

chiVal, pVal, df, expected = chi2_contingency(observed)

alpha = 0.05
print("p value is " + str(pVal)) 
if pVal <= alpha: 
    print('Dependent (reject H0)') 
else: 
    print('Independent (H0 holds true)')
    
N = observed.sum(axis = 1).sum()    

V = np.sqrt( (chiVal/N) / (min(observed.shape)-1) )
print("V: "+ str(V) + " a value close to 0 means weak, close to 1 means strong")
observed_pd

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 6))

light_labels = ["Daylight", "Darkness - lights lit", "Darkness - lights unlit", "Darkness - no lighting", "Darkness - lighting unknown"]
road_categories = {1: "Roundabout", 2: "One way street", 3: "Dual carriageway", 6: "Single carriageway", 7: "Slip road", 9: "Unknown"}
x = np.array(list(road_categories.keys()))

for i, ax in enumerate(axes[0]):
    ax.plot(x, observed[:,i], 'ro-', label='Observed')
    ax.plot(x, expected[:,i], 'bo-', label='Expected')
    if i==0: 
        ax.set_ylabel('Accidents')
        ax.legend(loc='best');
    ax.set_title(light_labels[i])
    ax.set_xticks(x)
    ax.set_xticklabels(list(road_categories.values()))
    fig.autofmt_xdate(rotation=45)

for i, ax in enumerate(axes[1]):
    ax.plot(x, observed[:,i]/expected[:,i], 'go-')
    ax.plot(x, np.ones(x.shape), 'k:')
    
    if i==0: 
        ax.set_ylabel('Observed/Expected')
    ax.set_xticks(x)
    ax.set_xticklabels(list(road_categories.values()))
    fig.autofmt_xdate(rotation=30)
    


# Task 3 - Map visualization
Visualize the reported collisions on a map of your city. Make a visual distinction between different
classes of casualty severity

In [ ]:
BBox = min(sheffield_data["accidents"]["Longitude"]), max(sheffield_data["accidents"]["Longitude"]), min(sheffield_data["accidents"]["Latitude"]),  max(sheffield_data["accidents"]["Latitude"])
ruh_m = plt.imread('../map.png')
mask_1 = (sheffield_data["accidents"]["Accident_Severity"] == 1)
mask_2 = (sheffield_data["accidents"]["Accident_Severity"] == 2)
mask_3 = (sheffield_data["accidents"]["Accident_Severity"] == 3)
fig, ax = plt.subplots(figsize = (16,14))

ax.scatter(sheffield_data["accidents"]["Longitude"][mask_1], sheffield_data["accidents"]["Latitude"][mask_1], alpha= 0.5, c='r', s=100)
ax.scatter(sheffield_data["accidents"]["Longitude"][mask_2], sheffield_data["accidents"]["Latitude"][mask_2], alpha= 0.5, c='y', s=90)
ax.scatter(sheffield_data["accidents"]["Longitude"][mask_3], sheffield_data["accidents"]["Latitude"][mask_3], alpha= 0.5, c='g', s=80)

ax.set_title('Plotting Spatial Data on Riyadh Map')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal');

# Task 4: Open question
Use the data to formulate, motivate, answer, and discuss another research question of your choice.
For example, compare your city to the whole UK, or investigate different collision participants (bicycle
vs car, bicycle-motored two-wheeler, pedestrian vs car, car vs car,…), or compare the age distribution
of casualties to the existing age distribution, or rank/compare how problematic different driver
demographics are (young/old male/female),…